# Deep Learning week - Day 3 - Exercise 3: Transfer Learning

The exercise one made you build your first Convolutional Neural Network.
In the second exercise, you implemented first data augmentation techniques to improve the accuracy of your model. 
But you've probably seen that when building complex convolutional neural networks and using a lot of data, the training started to take some time. 


# Google Colab


To overcome this long training time, you can train your neural networks (especially convolutional neural networks) on Google Colab. Google Colab is nothing more than a way to have online notebooks, with the possibility to use Google GPUs. The idea here is not to use it in production (as there are some limitations) but to use Google Colab to test and prototype new algorithms. This free access to GPU allows you to accelerate the computational time. 

So the first task is to open the notebook you are reading on Google Colab. To do so: 
- Open Google Colab [here](https://colab.research.google.com/)
- Import a Notebook and select this file, the one you are currently reading.
- Once open, you a running a similar Notebook but in Google Colab.
- Change the runtime type to GPU - default is CPU.

Welcome to Google Colab! Do not forget that you can open any notebook in Google Colab (but it is not always worth it and working on your computer might be enough for many tasks - however, with images, we recommand to do it on Google Colab. Moreover, we will mention the exercises that are preferably done on Colab during the week.).


# The exercise


This notebook is dedicated to **transfer learning** which corresponds to using an existing model that has been already trained on a particular task, and fine-tuning it for a different task.

To that end, we will use the [VGG-16 Neural Network](https://neurohive.io/en/popular-networks/vgg16/), a well-known architecture that has been trained on ImageNet which is a very large database of images of different categories. In a nutshell, this architecture has already learnt kernels which are supposed to be good not only for the task it has been train on but maybe for other tasks. 

The idea is that first layers are not specialized for the particular task it has been trained on ; only the last one are. Therefore, we will load the existing VGG16 network, remove the last fully connected layers, replace them by new connected layers (whose weights are randomly set), and train these last layers on a specific classification task - here, separate types of flower. The underlying idea is that the first convolutional layers of VGG-16, that has already been trained, corresponds to filters that are able to extract meaning features from images. And you will only learn the last layers for your particular problem.


# Data loading & Preprocessing

You have two options to load the data on Google Colab.


### Option 1: Loading the data directly 

You can first get the data onto google Colab thanks to:

`!wget https://wagon-public-datasets.s3.amazonaws.com/flowers-dataset.zip`,

and then run 

`!unzip flowers-dataset.zip`

This is a very easy option to load the data into your working directory.


### Option2: Adding the data to Google Drive.

You can first download the data  from `https://wagon-public-datasets.s3.amazonaws.com/flowers-dataset.zip`. Then you have to add them to your Google Drive in a folder called `Deep_learning_data` (for instance) and run the following code in the notebook.: 

```
from google.colab import drive
drive.mount('/content/drive/')
```

The previous code will ask you to go to a given webpage where you copy the link and past it in the Colab form that will appear. Do so to load the data on Google Colab.

Why choosing this option over the first one? This can be of interest if you work in a project team, and update the data from time to time. By doing this, you can share the same data folder within a team, and be sure that everyone has the same at any time, even though someone changes it. The drawback is that Google Collab has now access to your Google Folder, which you might not be in favor of, depending on your sensibility.

❓ **Question** ❓ Use one of the above method to load your data. 

In [ ]:
# YOUR CODE HERE

❓ **Question** ❓ Depending on the `loading_method` you have used, load the data and store them in the appropriate variables.

In [ ]:
from tqdm import tqdm
import numpy as np
import os
from PIL import Image

def load_flowers_data():
    data_path = '/content/drive/My Drive/06-Deep-Learning/02/data/flowers'
    classes = {'daisy':0, 'dandelion':1, 'rose':2}
    imgs = []
    labels = []
    for (cl, i) in classes.items():
        images_path = [elt for elt in os.listdir(os.path.join(data_path, cl)) if elt.find('.jpg')>0]
        for img in tqdm(images_path[:300]):
            path = os.path.join(data_path, cl, img)
            if os.path.exists(path):
                image = Image.open(path)
                image = image.resize((256, 256))
                imgs.append(np.array(image))
                labels.append(i)

    X = np.array(imgs)
    num_classes = len(set(labels))
    y = to_categorical(labels, num_classes)

    # Finally we shuffle:
    p = np.random.permutation(len(X))
    X, y = X[p], y[p]

    first_split = int(len(imgs) /6.)
    second_split = first_split + int(len(imgs) * 0.2)
    X_test, X_val, X_train = X[:first_split], X[first_split:second_split], X[second_split:]
    y_test, y_val, y_train = y[:first_split], y[first_split:second_split], y[second_split:]
    
    return X_train, y_train, X_val, y_val, X_test, y_test, num_classes


# YOUR CODE HERE

First things first, let's start by some preprocessing

❓ **Question** ❓ First check that all your images have the same size. How many colors do these images have?

In [ ]:
# YOUR CODE HERE

❓ **Question** ❓ Plot some images with the `imshow` function of matplotlib.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# YOUR PLOT HERE

# Baseline model


❓ **Question** ❓ As it is already the third exercise, let's make you work a bit on your own to create your own CNN. Here, you are free to write any CNN architecture that is trainable on the flower data. Do not forget to compile it and fit it on the train data (use the `validation_data` with the `X_val` and `y_val` instead of using the `validation_split` argument). Do not forget the early stopping criterion.

<details>
   <summary>(If you really do not know where to start from, you can build the following model:</summary>
    <ul>
        <li> A convolution layer with enough filters - choose the kernel size wisely as your data are quite large here</li>
        <li> A Maxpooling layer </li>
        <li> A second convolution layer </li>
        <li> A Maxpooling layer </li>
        <li> A third convolution layer </li>
        <li> A Maxpooling layer </li>
        <li> Flatten your output </li>
        <li> Add a dense layer </li>
        <li> Add the last layer </li>
        
</ul>
</details>



As you already know how to build CNN, you are here welcome to write your own model, compile it and fit it on the train data (use the 

In [ ]:
# YOUR CODE HERE

❓ **Question** ❓ Play a bit with your model to look at the effect of different architectures, especially by:
- changing the number of convolution layers (associated to their maxpooling layer)
- changing the number of filters
- changing the kernel sizes
- changing the padding and strides

In [ ]:
# YOUR CODE HERE

# Transfer learning: VGG16 model

Let's now build our model. 

❓ **Question** ❓ Write a first function `load_model()` that loads the pretrained VGG-16 model from `tensorflow.keras.applications.vgg16`. Especially, look at the [documentation](https://www.tensorflow.org/api_docs/python/tf/keras/applications/VGG16) to load the model where:
- the `weights` have been learnt on `imagenet`
- the `input_shape` corresponds to the input shape of any of your images - you have to resize them in case they are not of the same size
- the `include_top` argument is set to `False` in order not to load the fully-connected layers of the VGG-16 without the last layer which was specifically trained on `imagenet`

❗ **Remark** ❗ Do not change the default value of the other arguments

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

def load_model():
    
    # YOUR CODE HERE
    
    return model

❓ **Question** ❓ Look at the architecture of the model thanks to the summary method

In [ ]:
# YOUR CODE HERE

Impressive, right? Two things to notice:
- It ends with a convolution layer (namely a maxpooling layer that is the layer that follows a convolution). The flattening of the output and the fully connected layers are not here yet! We will add them
- There are more than 14.000.000 parameters, which is a lot. We could fine-tune them, meaning still updates them in the model algorithm, but it will take a lot of time, therefore we will fix them so they are not trainable. And only the last layers (that we will add) will be trained and updates

❓ **Question** ❓ Write a function that takes the previous model as input, and does what we just described:
- set the first layers to be not-trainable, by applying `model.trainable = False`
- add new layers (that by default will be trainable).

To do the second part (adding new layers), we will use another way to define the model (instead of `model.sequential()`). Look at the following code and try to apply it in your scenario:

```
base_model = load_model()
dense_layer = layers.Dense(SOME_NUMBER_1, activation='relu')
prediction_layer = layers.Dense(SOME_NUMBER_2, activation='APPROPRIATE_ACTIVATION')

model = tf.keras.Sequential([
  base_model,
  dense_layer,
  prediction_layer
])

```

❗ **Remark** ❗ Let's consider a dense layer with 500 neurons ; on the other hand, the prediction layer should be related to this specific task, so just replace `SOME_NUMBER_2` and `APPROPRIATE_ACTIVATION` with the correct values. 


❗ **Remark** ❗ Do not forget to Flatten the convoluted outputs first, before the fully connected layers ;)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models


def update_model(model):
    # Set the first layers to be untrainable
    # YOUR CODE HERE
    
    # Add layers to the mdoel
    # YOUR CODE HERE
    
    return model

❓ **Question** ❓ Now look at the layers and parameters of your model. Note that there is a distinction, at the end, between the trainable and non-trainable parameters

In [ ]:
# YOUR CODE HERE

❓ **Question** ❓ Write a function to compile your model - we advise Adam with `learning_rate=1e-4`. 

In [ ]:
from tensorflow.keras import optimizers

# YOUR CODE HERE

❓ **Question** ❓ Write an overall function that :
- load the model
- update the layers
- compiles it

In [ ]:

def build_model():
    # YOUR CODE HERE
    return model

model = build_model()

# Back to the data

The VGG16 model was trained on images which were preprocessed in a specific way. 

❓ **Question** ❓ Apply this processing to the images here using the method `preprocess_input` that you can import from `tensorflow.keras.applications.vgg16`.

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

# YOUR CODE HERE

# Run the model

❓ **Question** ❓ Now estimate the model, with an early stopping criterion on the validation accuracy - here, the validation data are provided, therefore use `validation_data` instead of `validation_split`.

❗ **Remark** ❗ Store the results in a `history` variable

In [ ]:
# YOUR CODE HERE

❓ **Question** ❓ Plot the accuracy for the test and validation set.

In [ ]:
# YOUR PLOT HERE

❓ **Question** ❓ Evaluate the model accuracy on the test set. What is the chance level on this classification task (i.e. accuracy of a random classifier).

In [ ]:
# YOUR CODE HERE

# Data augmentation

The next question are a less guided as they directly derive from what you have done in the previous exercise - don't hesitate to come back to what you have done.

❓ **Question** ❓ Use some data augmentation techniques for this task - you can store the fitting in a `history_data_aug` variable that you can plot. Do you see an improvement ? Don't forget to evaluate it on the test set

In [ ]:
# YOUR PLOT HERE

In [ ]:
# YOUR CODE HERE

# Improve the model

You can here try to improve the model test accuracy. To do that, here are some options you can consider

1) Is my model overfitting ? If yes, you can try more data augmentation. If no, try a more complex model (unlikely the case here)

2) Perform precise grid search on all the hyper-parameters: learning_rate, batch_size, data augmentation etc...

3) Change the base model to more modern one (resnet, efficient nets) available in the keras library

4) Curate the data: maintaining a sane data set is one of the keys to success.

5) Obtain more data


❗ **Remark** ❗ Note also that it is good practice to perform a real cross-validation. You can also try to do that here to be sure of your results.